# FINBERT Sentiment


# Data

In [ ]:
import os

dataset_files = [
    f for f in os.listdir('/content/')
    if f.endswith(('.csv', '.json'))
]

if dataset_files:
    for f in dataset_files:
        print(f)
else:
    print("No common dataset files (.csv, .json) found in /content/")

data_2015.json
FNHD_cnbc_headlines.csv
data_2016.json
data_2021.json
data_2022.json
data_2018.json
data_2017.json
sp500_headlines_2008_2024.csv
data_2020.json
data_2019.json
data_2013.json
data_2014.json
data_2012.json
FNHD_reuters_headlines.csv
FNHD_guardian_headlines.csv


# Natural Language CSVs Merge

In [ ]:
import pandas as pd
import json

def count_rows_after_dropping_na(filepath, file_type):
    if file_type == 'json':
        with open(filepath, 'r') as f:
            data = json.load(f)
        df = pd.DataFrame(data)
    elif file_type == 'csv':
        df = pd.read_csv(filepath)
    else:
        print(f"Unsupported file type for {filepath}")
        return 0

    original_rows = len(df)
    df_cleaned = df.dropna()
    cleaned_rows = len(df_cleaned)
    print(f"File: {filepath}\n  Original rows: {original_rows}\n  Rows after dropping missing values: {cleaned_rows}\n")
    return cleaned_rows

# List of files to process with their types, excluding data_2013.json
files_to_process = {
    'FNHD_reuters_headlines.csv': 'csv',
    'sp500_headlines_2008_2024.csv': 'csv',
    'FNHD_guardian_headlines.csv': 'csv',
    'FNHD_cnbc_headlines.csv': 'csv'
}

total_cleaned_rows = 0
for file_path, file_type in files_to_process.items():
    total_cleaned_rows += count_rows_after_dropping_na(file_path, file_type)

print(f"Total rows after dropping missing values across all files (excluding data_2013.json): {total_cleaned_rows}")

File: FNHD_reuters_headlines.csv
  Original rows: 32770
  Rows after dropping missing values: 32770

File: sp500_headlines_2008_2024.csv
  Original rows: 19127
  Rows after dropping missing values: 19127

File: FNHD_guardian_headlines.csv
  Original rows: 17800
  Rows after dropping missing values: 17800

File: FNHD_cnbc_headlines.csv
  Original rows: 3080
  Rows after dropping missing values: 2800

Total rows after dropping missing values across all files (excluding data_2013.json): 72497


In [ ]:
# ============================================================
#  Merge Financial News CSVs (Full Chronological Version — Deduplication Safe)
# ============================================================

import pandas as pd
import re
from datetime import datetime

# ---------- 1️⃣ S&P 500 ----------
sp500 = pd.read_csv("sp500_headlines_2008_2024.csv")
sp500 = sp500.rename(columns={"Title": "headline", "Date": "date", "CP": "close"})
sp500 = sp500[["date", "headline"]].dropna()
sp500["description"] = None
sp500["date"] = pd.to_datetime(sp500["date"], format="%Y-%m-%d", errors="coerce")
print(f"✅ Loaded sp500: {len(sp500):,} rows")

# ---------- 2️⃣ Guardian ----------
guardian = pd.read_csv("FNHD_guardian_headlines.csv")
guardian = guardian.rename(columns={"Headlines": "headline", "Time": "date"})
guardian = guardian[["date", "headline"]].dropna(subset=["headline"])
guardian["description"] = None

guardian["date"] = guardian["date"].astype(str).str.strip()

# Repair partial month-year patterns (e.g. "Jul-18")
mask_partial = guardian["date"].str.match(r"^[A-Za-z]{3,9}[-\s]\d{2,4}$", na=False)
num_partial = mask_partial.sum()
guardian.loc[mask_partial | guardian["date"].str.len().lt(3), "date"] = pd.NA
guardian["date"] = guardian["date"].ffill().bfill()
guardian["date"] = pd.to_datetime(guardian["date"], format="%d-%b-%y", errors="coerce")
print(f"✅ Loaded guardian: {len(guardian):,} rows | repaired {num_partial:,}")

# ---------- 3️⃣ Reuters ----------
reuters = pd.read_csv("FNHD_reuters_headlines.csv")
reuters = reuters.rename(columns={"Headlines": "headline", "Time": "date", "Description": "description"})
reuters = reuters[["date", "headline", "description"]].dropna(subset=["headline"])
reuters["date"] = pd.to_datetime(reuters["date"], format="%b %d %Y", errors="coerce")

def flexible_reuters_date(x):
    match = re.search(r"([A-Za-z]{3,9})\s+(\d{1,2})\s+(\d{4})", str(x))
    if match:
        for fmt in ("%B %d %Y", "%b %d %Y"):
            try:
                return datetime.strptime(" ".join(match.groups()), fmt)
            except:
                continue
    return pd.NaT

reuters.loc[reuters["date"].isna(), "date"] = reuters.loc[
    reuters["date"].isna(), "date"
].apply(flexible_reuters_date)
print(f"✅ Loaded reuters: {len(reuters):,} rows | invalid dates: {reuters['date'].isna().sum():,}")

# ---------- 4️⃣ CNBC ----------
cnbc_raw = pd.read_csv("FNHD_cnbc_headlines.csv")
subset_needed = ["Headlines", "Time", "Description"]
missing_required = cnbc_raw[subset_needed].isna().any(axis=1)

print(f"\n⚠️ CNBC rows missing any of {subset_needed}: {missing_required.sum():,}")
print("🚫 Example dropped rows (first 5):")
print(cnbc_raw[missing_required].head(5))

cnbc = cnbc_raw.rename(columns={"Headlines": "headline", "Time": "date", "Description": "description"})
cnbc = cnbc.dropna(subset=["headline", "date"], how="all").copy()

def extract_cnbc_date(text):
    if not isinstance(text, str) or not text.strip():
        return pd.NaT
    text = re.sub(r"\s+", " ", text.strip()).replace("Sept", "Sep")
    match = re.search(r"(\d{1,2})\s+([A-Za-z]+)\s+(\d{4})", text)
    if match:
        day, month, year = match.groups()
        for fmt in ("%d %B %Y", "%d %b %Y"):
            try:
                return datetime.strptime(f"{day} {month} {year}", fmt)
            except ValueError:
                continue
    return pd.NaT

cnbc["date"] = cnbc["date"].apply(extract_cnbc_date)
invalid_count = cnbc["date"].isna().sum()
print(f"✅ Loaded cnbc: {len(cnbc):,} rows | invalid dates: {invalid_count:,}")

# ---------- 5️⃣ Combine & Tag ----------
sp500["source"] = "sp500"
guardian["source"] = "guardian"
reuters["source"] = "reuters"
cnbc["source"] = "cnbc"

merged_raw = pd.concat([sp500, guardian, reuters, cnbc], ignore_index=True)
print(f"\n🧩 Combined raw rows: {len(merged_raw):,}")

# Drop NA dates/headlines
merged_raw = merged_raw.dropna(subset=["date", "headline"])

# ---------- 6️⃣ Deduplication & Audit ----------
before = len(merged_raw)
deduped = merged_raw.drop_duplicates(subset=["date", "headline"], keep="first").copy()
after = len(deduped)
dupes_removed = before - after

print(f"\n🧮 Deduplication summary:")
print(f"Raw rows before: {before:,}")
print(f"Rows after deduplication: {after:,}")
print(f"Rows removed: {dupes_removed:,}")

# Inspect where duplicates came from
dupes = merged_raw[merged_raw.duplicated(subset=["date", "headline"], keep=False)]
if len(dupes) > 0:
    print(f"\n📑 Duplicate rows found: {len(dupes):,}")
    print("Example duplicates:")
    print(dupes.sort_values("date").head(10)[["date", "headline", "source"]])
else:
    print("✅ No duplicates detected.")

# ---------- 7️⃣ Verify Chronology ----------
deduped = deduped.sort_values("date").reset_index(drop=True)
deduped["date"] = deduped["date"].dt.strftime("%m/%d/%Y")

# ---------- 8️⃣ Save Clean File ----------
deduped.to_csv("financial_headlines_full.csv", index=False)

print(f"\n✅ Final dataset saved → financial_headlines_full.csv")
print(f"Total rows: {len(deduped):,}")
print(f"Period: {deduped['date'].iloc[0]} → {deduped['date'].iloc[-1]}")

# ---------- 9️⃣ Diagnostic Summary ----------
summary = deduped["source"].value_counts()
print("\n📊 Final composition by source:")
print(summary)


✅ Loaded sp500: 19,127 rows
✅ Loaded guardian: 17,800 rows | repaired 40
✅ Loaded reuters: 32,770 rows | invalid dates: 0

⚠️ CNBC rows missing any of ['Headlines', 'Time', 'Description']: 280
🚫 Example dropped rows (first 5):
   Headlines Time Description
2        NaN  NaN         NaN
9        NaN  NaN         NaN
24       NaN  NaN         NaN
31       NaN  NaN         NaN
46       NaN  NaN         NaN
✅ Loaded cnbc: 2,800 rows | invalid dates: 0

🧩 Combined raw rows: 72,497

🧮 Deduplication summary:
Raw rows before: 72,497
Rows after deduplication: 71,412
Rows removed: 1,085

📑 Duplicate rows found: 2,157
Example duplicates:
           date                                           headline source
1139 2012-03-13     Grenade-throwing robot to fight fires on ships  sp500
1141 2012-03-13     Grenade-throwing robot to fight fires on ships  sp500
1556 2012-10-04   Stocks for the Long Run: Hershey vs. the S&P 500  sp500
1559 2012-10-04   Stocks for the Long Run: Hershey vs. the S&P 500  s

In [ ]:
# ============================================================
# 🔍 Detect All Duplicate Headline Clusters (≥2 Occurrences)
# ============================================================

import pandas as pd

# Combine all sources into one DataFrame for full duplicate inspection
dupes_all = pd.concat([sp500, guardian, reuters, cnbc], ignore_index=True)
dupes_all["date"] = pd.to_datetime(dupes_all["date"], errors="coerce")

# Count occurrences of each (date, headline)
dup_counts = (
    dupes_all.groupby(["date", "headline"], dropna=False)
    .size()
    .reset_index(name="count")
    .sort_values("count", ascending=False)
)

# Filter all that appear 2 or more times (duplicates, triples, quadruples, etc.)
multi_dupes = dup_counts[dup_counts["count"] >= 2]

print(f"\n⚠️ Duplicated (date, headline) pairs found: {len(multi_dupes):,}")
if not multi_dupes.empty:
    print("\n🧾 Example duplicates (top 15):")
    print(multi_dupes.head(15))
else:
    print("✅ No duplicates detected.")

# ---- Inspect which datasets contributed to these duplicates ----
merged_sources = pd.concat(
    [
        sp500.assign(source="sp500"),
        guardian.assign(source="guardian"),
        reuters.assign(source="reuters"),
        cnbc.assign(source="cnbc")
    ],
    ignore_index=True
)

dupe_sources = merged_sources.merge(
    multi_dupes[["date", "headline"]],
    on=["date", "headline"],
    how="inner"
).sort_values("date")

print(f"\n📊 Total duplicate rows across sources: {len(dupe_sources):,}")
print("🔎 Example cross-source duplicates (first 10):")
print(dupe_sources.head(10)[["date", "headline", "source"]])



⚠️ Duplicated (date, headline) pairs found: 1,072

🧾 Example duplicates (top 15):
            date                                           headline  count
44775 2019-11-21         What is the S&P 500 Average Annual Return?      4
30877 2019-04-08  S&P 500, Nasdaq edge higher as earnings season...      3
25088 2019-01-08  Apple, Facebook propel Wall Street to three-we...      3
16190 2018-07-31  Apple buybacks eclipse value of most S&P 500 c...      3
37969 2019-08-07  S&P 500 erases big losses to end up; investors...      3
43368 2019-11-01  S&P 500, Nasdaq set records on jobs data, trad...      3
46721 2019-12-27  U.S. energy shareholders seek to leave behind ...      3
38527 2019-08-19  Wall Street rallies on hopes of global economi...      3
46614 2019-12-23                 S&P is the decade's runaway winner      3
46278 2019-12-17  Wall Street extends record-setting climb on up...      3
32124 2019-04-29  Global stocks climb as S&P 500 hits record, oi...      3
46813 2019-12-31 

# Natural Language CSVs + JSON Merge

In [ ]:
# ============================================================
# 🧩 Combine All Financial News JSONs into Unified CSV (Clean + Deduped)
# ============================================================

import json
import pandas as pd
from glob import glob
from datetime import datetime

# --- 1️⃣ Find all JSON files ---
json_files = sorted(glob("data_*.json"))  # expects files like data_2012.json ... data_2022.json
print("📦 Found JSON files:")
for f in json_files:
    print(f"  • {f}")

# --- 2️⃣ Load & Normalize ---
dfs = []
for file in json_files:
    try:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)
        if not isinstance(data, list):
            print(f"⚠️ {file}: Unexpected root structure ({type(data)}) — skipping.")
            continue
        if len(data) == 0:
            print(f"⚠️ {file}: Empty list — skipping.")
            continue

        df = pd.DataFrame(data)
        cols = [c.lower() for c in df.columns]

        # Identify key columns
        date_col = next((c for c in cols if "date" in c), None)
        headline_col = next((c for c in cols if "headline" in c), None)
        desc_col = next((c for c in cols if "short" in c or "desc" in c), None)

        # Standardize columns
        df = df.rename(columns={
            date_col: "date",
            headline_col: "headline",
            desc_col: "description"
        })
        df = df[["date", "headline", "description"]]

        # Drop missing rows
        df = df.dropna(subset=["headline", "date"])
        df["description"] = df["description"].fillna("")

        # Convert to datetime
        df["date"] = pd.to_datetime(df["date"], errors="coerce")
        df = df.dropna(subset=["date"])

        dfs.append(df)
        print(f"✅ {file}: loaded {len(df):,} valid rows.")

    except Exception as e:
        print(f"❌ Error reading {file}: {e}")

# --- 3️⃣ Combine ---
json_all = pd.concat(dfs, ignore_index=True)
json_all = json_all.sort_values("date").reset_index(drop=True)
json_all["date"] = json_all["date"].dt.strftime("%m/%d/%Y")

# --- 4️⃣ Detect duplicates before saving ---
dupes = json_all[json_all.duplicated(subset=["date", "headline"], keep=False)]
print(f"\n🔎 Found {len(dupes):,} potential duplicates (same date + headline).")

if len(dupes) > 0:
    print("\n📋 Example duplicates (first 10):")
    print(dupes.head(10)[["date", "headline"]])

# Drop duplicates (keep first occurrence)
json_all = json_all.drop_duplicates(subset=["date", "headline"]).reset_index(drop=True)

# --- 5️⃣ Add source column ---
json_all["source"] = "harvard"

# --- 6️⃣ Save ---
json_all.to_csv("financial_headlines_jsons.csv", index=False)

print("\n✅ Combined JSON dataset saved → financial_headlines_jsons.csv")
print(f"Total rows after cleaning: {len(json_all):,}")
print(f"Period: {json_all['date'].iloc[0]} → {json_all['date'].iloc[-1]}")

print("\n📊 Summary:")
try:
    print(json_all.describe(include='all', datetime_is_numeric=True))
except TypeError:
    print(json_all.describe(include='all'))

print("\n🧾 Preview:")
print(json_all.head(10))



📦 Found JSON files:
  • data_2012.json
  • data_2013.json
  • data_2014.json
  • data_2015.json
  • data_2016.json
  • data_2017.json
  • data_2018.json
  • data_2019.json
  • data_2020.json
  • data_2021.json
  • data_2022.json
✅ data_2012.json: loaded 31,353 valid rows.
✅ data_2013.json: loaded 34,583 valid rows.
✅ data_2014.json: loaded 32,339 valid rows.
✅ data_2015.json: loaded 32,006 valid rows.
✅ data_2016.json: loaded 32,098 valid rows.
✅ data_2017.json: loaded 29,889 valid rows.
✅ data_2018.json: loaded 9,734 valid rows.
✅ data_2019.json: loaded 2,005 valid rows.
✅ data_2020.json: loaded 2,054 valid rows.
✅ data_2021.json: loaded 2,066 valid rows.
✅ data_2022.json: loaded 1,398 valid rows.

🔎 Found 843 potential duplicates (same date + headline).

📋 Example duplicates (first 10):
            date                                           headline
345   02/01/2012            World's Most Popular Landmarks (PHOTOS)
355   02/01/2012            World's Most Popular Landmarks (PHOT

In [ ]:
# ============================================================
# 🔗 Merge Financial Headlines (CSV sources + JSON sources)
# ============================================================

import pandas as pd

# --- 1️⃣ Load both datasets ---
csv_part = pd.read_csv("financial_headlines_csvs.csv")
json_part = pd.read_csv("financial_headlines_jsons.csv")

print(f"✅ CSV dataset: {len(csv_part):,} rows")
print(f"✅ JSON dataset: {len(json_part):,} rows")

# --- 2️⃣ Add source column to JSON data ---
json_part["source"] = "harvard"   # consistent label for HuffPost/JSON data

# --- 3️⃣ Ensure consistent columns ---
csv_part = csv_part[["date", "headline", "description", "source"]]
json_part = json_part[["date", "headline", "description", "source"]]

# --- 4️⃣ Combine ---
combined = pd.concat([csv_part, json_part], ignore_index=True)

# Drop missing headlines or dates just in case
combined = combined.dropna(subset=["date", "headline"])

# --- 5️⃣ Convert and sort ---
combined["date"] = pd.to_datetime(combined["date"], errors="coerce")
combined = combined.dropna(subset=["date"])
combined = combined.sort_values("date").reset_index(drop=True)

# --- 6️⃣ Save ---
combined["date"] = combined["date"].dt.strftime("%m/%d/%Y")
combined.to_csv("financial_headlines_master.csv", index=False)

# --- 7️⃣ Summary ---
print("\n✅ Final master dataset saved → financial_headlines_master.csv")
print(f"Total rows: {len(combined):,}")
print(f"Period: {combined['date'].iloc[0]} → {combined['date'].iloc[-1]}")
print("\n📊 Sources breakdown:")
print(combined["source"].value_counts())
print("\n🧾 Preview:")
print(combined.head(10))


✅ CSV dataset: 71,412 rows
✅ JSON dataset: 209,099 rows

✅ Final master dataset saved → financial_headlines_master.csv
Total rows: 280,505
Period: 01/02/2008 → 03/04/2024

📊 Sources breakdown:
source
harvard     209093
reuters      32673
sp500        18153
guardian     17799
cnbc          2787
Name: count, dtype: int64

🧾 Preview:
         date                                           headline description  \
0  01/02/2008   JPMorgan Predicts 2008 Will Be "Nothing But Net"         NaN   
1  01/02/2008  Dow Tallies Biggest First-session-of-year Poin...         NaN   
2  01/02/2008                   2008 predictions for the S&P 500         NaN   
3  01/03/2008  U.S. Stocks Higher After Economic Data, Monsan...         NaN   
4  01/07/2008  U.S. Stocks Climb As Hopes Increase For More F...         NaN   
5  01/09/2008  How Investing in Intangibles -- Like Employee ...         NaN   
6  01/09/2008         Head And Shoulders Top Bodes Ill For Bulls         NaN   
7  01/10/2008  U.S. Stocks 

In [ ]:
# Compare before/after merge for Harvard JSON rows
json_part["pair"] = list(zip(json_part["date"], json_part["headline"]))
merged_json = combined[combined["source"] == "harvard"].copy()
merged_json["pair"] = list(zip(merged_json["date"], merged_json["headline"]))

dropped_json = json_part.loc[~json_part["pair"].isin(merged_json["pair"])]
print(f"🧾 Dropped Harvard rows: {len(dropped_json):,}")
if len(dropped_json) > 0:
    print("\n🔎 Examples:")
    print(dropped_json.head(10)[["date", "headline"]])


🧾 Dropped Harvard rows: 6

🔎 Examples:
              date headline
86947   08/28/2014      NaN
98963   01/13/2015      NaN
100007  01/25/2015      NaN
105422  03/29/2015      NaN
113595  06/30/2015      NaN
118250  08/22/2015      NaN


In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("financial_headlines_master.csv")
print(f"✅ Loaded master dataset: {len(df):,} rows")

# Convert 'date' to datetime (handles mm/dd/yyyy format)
df["date"] = pd.to_datetime(df["date"], errors="coerce", format="%m/%d/%Y")

# Identify duplicates (same date + headline)
dupes = df[df.duplicated(subset=["date", "headline"], keep=False)]
print(f"\n🔎 Total duplicate rows (same date + headline): {len(dupes):,}")

if len(dupes) > 0:
    print("\n📋 Example duplicates (first 10):")
    print(dupes.head(10)[["date", "headline", "source"]])

    # --- Group by year & source ---
    dupe_summary = (
        dupes.groupby([dupes["date"].dt.year, "source"])
        .size()
        .reset_index(name="duplicate_count")
        .sort_values("duplicate_count", ascending=False)
    )

    print("\n📊 Duplicate summary by year & source:")
    print(dupe_summary.head(20))



✅ Loaded master dataset: 280,505 rows

🔎 Total duplicate rows (same date + headline): 0


# FINBERT Implementation

In [ ]:
# ============================================================
# ⚙️ FinBERT Sentiment Analysis on Financial Headlines
# ============================================================

import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

# ------------------------------------------------------------
# 1️⃣ Load master dataset
# ------------------------------------------------------------
print("📥 Loading master dataset ...")
df = pd.read_csv("financial_headlines_master.csv")

# Parse and clean
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df.dropna(subset=["date", "headline"])
df["text"] = df["headline"].fillna("") + " " + df["description"].fillna("")
print(f"✅ Loaded {len(df):,} rows with valid text.\n")

# ------------------------------------------------------------
# 2️⃣ Load FinBERT model
# ------------------------------------------------------------
MODEL_NAME = "yiyanghkust/finbert-tone"
print(f"⚙️ Loading FinBERT model: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"💻 Using device: {device}\n")

# ------------------------------------------------------------
# 3️⃣ Run inference batch-wise
# ------------------------------------------------------------
batch_size = 32
probs_list = []

for i in tqdm(range(0, len(df), batch_size), desc="Running FinBERT"):
    batch_texts = df["text"].iloc[i:i+batch_size].tolist()
    tokens = tokenizer(
        batch_texts,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=128
    ).to(device)

    with torch.no_grad():
        outputs = model(**tokens)
        probs = F.softmax(outputs.logits, dim=-1).cpu().numpy()
    probs_list.extend(probs)

    # Optional checkpoint every 25k rows (safety for long Colab runs)
    if (i // batch_size) % 800 == 0 and i > 0:
        pd.DataFrame(probs_list, columns=["bearish", "neutral", "bullish"]).to_csv(
            f"finbert_partial_{i}.csv", index=False
        )
        print(f"💾 Checkpoint saved at row {i:,}")

# ------------------------------------------------------------
# 4️⃣ Attach predictions
# ------------------------------------------------------------
probs_df = pd.DataFrame(probs_list, columns=["bearish", "neutral", "bullish"])
df = pd.concat([df.reset_index(drop=True), probs_df], axis=1)

# ------------------------------------------------------------
# 5️⃣ Compute continuous sentiment index
# ------------------------------------------------------------
df["sentiment_index"] = df["bullish"] - df["bearish"]

# Aggregate daily means
daily = (
    df.groupby("date")[["bullish", "bearish", "neutral", "sentiment_index"]]
    .mean()
    .reset_index()
)

# Optional: rescale sentiment_index from (-1, 1) → (0, 1)
daily["sentiment_scaled"] = (daily["sentiment_index"] + 1) / 2

# Optional: 7-day moving average smoothing
daily["sentiment_smoothed"] = daily["sentiment_index"].rolling(7, min_periods=3).mean()

daily = daily.sort_values("date")

# ------------------------------------------------------------
# 6️⃣ Save results
# ------------------------------------------------------------
daily.to_csv("daily_finbert_sentiment.csv", index=False)

print("\n✅ Saved daily sentiment index → daily_finbert_sentiment.csv")
print(f"Total trading days analyzed: {len(daily):,}")
print(f"Period: {daily['date'].iloc[0].date()} → {daily['date'].iloc[-1].date()}")
print("\n🧾 Preview:")
print(daily.head(10))


📥 Loading master dataset ...
✅ Loaded 280,505 rows with valid text.

⚙️ Loading FinBERT model: yiyanghkust/finbert-tone


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

...

💾 Checkpoint saved at row 230,400


Running FinBERT:  91%|█████████▏| 8002/8766 [17:36<05:50,  2.18it/s]

💾 Checkpoint saved at row 256,000


Running FinBERT: 100%|██████████| 8766/8766 [18:58<00:00,  7.70it/s]



✅ Saved daily sentiment index → daily_finbert_sentiment.csv
Total trading days analyzed: 4,807
Period: 2008-01-02 → 2024-03-04

🧾 Preview:
        date       bullish   bearish   neutral  sentiment_index  \
0 2008-01-02  1.594703e-01  0.677818  0.162712        -0.518348   
1 2008-01-03  4.519582e-08  0.000001  0.999999        -0.000001   
2 2008-01-07  2.223817e-05  0.016125  0.983853        -0.016103   
3 2008-01-09  4.923710e-01  0.499714  0.007915        -0.007343   
4 2008-01-10  1.950251e-03  0.877425  0.120625        -0.875475   
5 2008-01-22  2.468518e-01  0.503019  0.250130        -0.256167   
6 2008-01-29  2.938654e-02  0.970608  0.000005        -0.941222   
7 2008-01-30  8.799168e-05  0.995723  0.004189        -0.995635   
8 2008-02-01  9.999925e-01  0.000004  0.000003         0.999988   
9 2008-02-05  6.666574e-01  0.333045  0.000297         0.333612   

   sentiment_scaled  sentiment_smoothed  
0          0.240826                 NaN  
1          0.499999                 Na

In [ ]:
# ============================================================
# ⚙️ FinBERT Sentiment Analysis (Daily + Weekly + Monthly)
# ============================================================

import pandas as pd
import torch
import torch.nn.functional as F
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ------------------------------------------------------------
# Helper function
# ------------------------------------------------------------
def run_finbert_sentiment(input_csv, output_csv_prefix):
    print(f"\n📥 Loading dataset: {input_csv}")
    df = pd.read_csv(input_csv)
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date", "headline"])

    # Combine text
    df["text"] = df["headline"].fillna("") + " " + df["description"].fillna("")
    df = df[df["text"].str.strip() != ""]
    print(f"✅ {len(df):,} rows with valid text")

    # --------------------------------------------------------
    # Load FinBERT
    # --------------------------------------------------------
    MODEL_NAME = "yiyanghkust/finbert-tone"
    print(f"\n⚙️ Loading FinBERT: {MODEL_NAME}")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"💻 Using device: {device}")
    model.to(device)
    model.eval()

    # --------------------------------------------------------
    # Batch inference
    # --------------------------------------------------------
    batch_size = 32
    probs_list = []

    for i in tqdm(range(0, len(df), batch_size), desc="Running FinBERT"):
        texts = df["text"].iloc[i:i+batch_size].tolist()
        enc = tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            out = model(**enc)
            probs = F.softmax(out.logits, dim=-1).cpu().numpy()

        probs_list.extend(probs)

    # --------------------------------------------------------
    # Attach bullish/bearish/neutral
    # --------------------------------------------------------
    probs_df = pd.DataFrame(probs_list, columns=["bearish", "neutral", "bullish"])
    df = pd.concat([df.reset_index(drop=True), probs_df], axis=1)

    # --------------------------------------------------------
    # Daily averages
    # --------------------------------------------------------
    daily = (
        df.groupby("date")[["bullish", "bearish", "neutral"]]
        .mean()
        .reset_index()
    )

    # --------------------------------------------------------
    # Build sentiment indexes
    # --------------------------------------------------------
    daily["sentiment_index"] = daily["bullish"] - daily["bearish"]
    daily["sentiment_scaled"] = (daily["sentiment_index"] + 1) / 2

    # Rolling windows:
    daily["sentiment_smoothed_7d"] = (
        daily["sentiment_scaled"].rolling(7, min_periods=1).mean()
    )
    daily["sentiment_smoothed_30d"] = (
        daily["sentiment_scaled"].rolling(30, min_periods=1).mean()
    )

    daily = daily.sort_values("date")

    # --------------------------------------------------------
    # Save
    # --------------------------------------------------------
    out_csv = f"{output_csv_prefix}_sentiment.csv"
    daily.to_csv(out_csv, index=False)

    print(f"\n✅ Saved sentiment to: {out_csv}")
    print(f"📅 Range: {daily['date'].iloc[0].date()} → {daily['date'].iloc[-1].date()}")
    print("\n🧾 Preview:")
    print(daily.head(10))

    return daily


# ============================================================
# 1️⃣ Run on MASTER dataset
# ============================================================
sent_master = run_finbert_sentiment(
    "financial_headlines_master.csv",
    "finbert_master"
)

# ============================================================
# 2️⃣ Run on CSV-only dataset
# ============================================================
sent_csvs = run_finbert_sentiment(
    "financial_headlines_csvs.csv",
    "finbert_csvs"
)



📥 Loading dataset: financial_headlines_master.csv
✅ 280,505 rows with valid text

⚙️ Loading FinBERT: yiyanghkust/finbert-tone
💻 Using device: cuda


Running FinBERT: 100%|██████████| 8766/8766 [18:54<00:00,  7.73it/s]



✅ Saved sentiment to: finbert_master_sentiment.csv
📅 Range: 2008-01-02 → 2024-03-04

🧾 Preview:
        date       bullish   bearish   neutral  sentiment_index  \
0 2008-01-02  1.594703e-01  0.677818  0.162712        -0.518348   
1 2008-01-03  4.519582e-08  0.000001  0.999999        -0.000001   
2 2008-01-07  2.223817e-05  0.016125  0.983853        -0.016103   
3 2008-01-09  4.923710e-01  0.499714  0.007915        -0.007343   
4 2008-01-10  1.950251e-03  0.877425  0.120625        -0.875475   
5 2008-01-22  2.468518e-01  0.503019  0.250130        -0.256167   
6 2008-01-29  2.938654e-02  0.970608  0.000005        -0.941222   
7 2008-01-30  8.799168e-05  0.995723  0.004189        -0.995635   
8 2008-02-01  9.999925e-01  0.000004  0.000003         0.999988   
9 2008-02-05  6.666574e-01  0.333045  0.000297         0.333612   

   sentiment_scaled  sentiment_smoothed_7d  sentiment_smoothed_30d  
0          0.240826               0.240826                0.240826  
1          0.499999        

Running FinBERT: 100%|██████████| 2232/2232 [04:18<00:00,  8.63it/s]



✅ Saved sentiment to: finbert_csvs_sentiment.csv
📅 Range: 2008-01-02 → 2024-03-04

🧾 Preview:
        date       bullish   bearish   neutral  sentiment_index  \
0 2008-01-02  1.594703e-01  0.677818  0.162712        -0.518348   
1 2008-01-03  4.519582e-08  0.000001  0.999999        -0.000001   
2 2008-01-07  2.223817e-05  0.016125  0.983853        -0.016103   
3 2008-01-09  4.923710e-01  0.499714  0.007915        -0.007343   
4 2008-01-10  1.950251e-03  0.877425  0.120625        -0.875475   
5 2008-01-22  2.468518e-01  0.503019  0.250130        -0.256167   
6 2008-01-29  2.938654e-02  0.970608  0.000005        -0.941222   
7 2008-01-30  8.799168e-05  0.995723  0.004189        -0.995635   
8 2008-02-01  9.999925e-01  0.000004  0.000003         0.999988   
9 2008-02-05  6.666574e-01  0.333045  0.000297         0.333612   

   sentiment_scaled  sentiment_smoothed_7d  sentiment_smoothed_30d  
0          0.240826               0.240826                0.240826  
1          0.499999          

# AI Appendix

Portions of this project were developed with assistance from AI tools (ChatGPT). AI was used primarily for debugging help, code structuring ideas, generating boilerplate code patterns, and clarifying Python, pandas, and TensorFlow syntax. All core logic, including data engineering design, anchor-based feature calculations, trade-segmentation framework, model selection, and evaluation methodology was designed, implemented, and validated by our team. Any AI-generated code snippets were reviewed, modified, and integrated manually to ensure correctness, interpretability, and alignment with our methodological goals. All conceptual decisions, modeling choices, and analysis were performed by the authors.